In [1]:
import irradiance_beds
import pandas as pd
import xarray as xr
import numpy as np

import pvdeg

In [52]:
pvdeg.geospatial.start_dask()

coords = [
    (37.01, -108.9),
    (37.01, -108.74)
]

# # this is to fix the missing weather data
weather, meta, failed = pvdeg.weather.weather_distributed(
    coords = coords,
    database="PSM3",
    email="tobin.ford@nrel.gov"
)

/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37111 instead
  warnings.warn(


Dashboard: http://127.0.0.1:37111/status
Connected to a Dask scheduler | Dashboard: http://127.0.0.1:37111/status


/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/pvlib/iotools/psm3.py:360: pvlibDeprecationWarning: PSM3 variable names will be renamed to pvlib conventions by default starting in pvlib 0.11.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(
/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/pvlib/iotools/psm3.py:360: pvlibDeprecationWarning: PSM3 variable names will be renamed to pvlib conventions by default starting in pvlib 0.11.0. Specify map_variables=True to enable that behavior now, or specify map_variables=False to hide this warning.
  warnings.warn(


Force load a sample output so we can generate an output template.

In [53]:
conf = "01"
fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"

loaded = xr.open_dataset(fp)
loaded_sub = loaded.isel(latitude=slice(0,1), longitude=range(1,3))

template = irradiance_beds.output_template(loaded_sub)

In [54]:
template.compute()

<xarray.Dataset> Size: 4MB
Dimensions:            (latitude: 1, longitude: 2, time: 8760,
                        distance_index: 10)
Coordinates:
  * latitude           (latitude) float32 4B 37.01
  * longitude          (longitude) float32 8B -108.9 -108.7
  * distance_index     (distance_index) int64 80B 0 1 2 3 4 5 6 7 8 9
  * time               (time) datetime64[ns] 70kB 2001-01-01T00:30:00 ... 200...
Data variables: (12/16)
    temp_air           (latitude, longitude, time) float64 140kB 6.938e-310 ....
    dew_point          (latitude, longitude, time) float64 140kB 6.938e-310 ....
    dhi                (latitude, longitude, time) float64 140kB 6.938e-310 ....
    dni                (latitude, longitude, time) float64 140kB 6.938e-310 ....
    ghi                (latitude, longitude, time) float64 140kB 6.938e-310 ....
    albedo             (latitude, longitude, time) float64 140kB 6.938e-310 ....
    ...                 ...
    ground_irradiance  (time, distance_index, latitude, longitude) float64 1MB ...
    underpanel         (latitude, longitude, time) float64 140kB 6.938e-310 ....
    edgetoedge         (latitude, longitude, time) float64 140kB 6.938e-310 ....
    bedA               (latitude, longitude, time) float64 140kB 6.938e-310 ....
    bedB               (latitude, longitude, time) float64 140kB 6.938e-310 ....
    bedC               (latitude, longitude, time) float64 140kB 6.938e-310 ....

In [50]:
for conf in ["01", "02", "03"]:

    conf_template = template.copy()
    
    for i in range(1,3):

        single_location_irrradiance_da = loaded.isel(latitude=0, longitude=i).ground_irradiance
        latitude, longitude = coords[i - 1]

        conf_beds = irradiance_beds.testbeds_irradiance(
            irradiance_df=single_location_irrradiance_da.to_pandas(),
            conf=conf
        )

        # save conf_beds results
        for var_name, da in conf_beds_ds.data_vars.items():
            conf_template[var_name].loc[dict(latitude=[latitude], longitude=[longitude])] = da

        # save irradiance data with distance axis
        conf_template['ground_irradiance'].loc[dict(latitude=[latitude], longitude=[longitude])] = single_location_irrradiance_da

        # save weather data|
        weather_da = weather.isel(gid=i - 1).drop_vars(["Year", "Month", "Day", "Hour", "Minute", "gid"])
        for var_name, da in weather_da.data_vars.items():
            conf_template[var_name].loc[dict(latitude=[latitude], longitude=[longitude])] = da

    # conf_template.to_netcdf(f"conf_{conf}-agrivoltaics-irradiance-weather.h5", engine="h5netcdf")

ValueError: new dimensions ('time', 'distance_index', 'latitude', 'longitude') must be a superset of existing dimensions ('time', 'distance')

In [26]:
#weather["time"] = res.index # overwrite mismatched time index, both are localized tmy data

for conf in ["01", "02", "03"]:
    for i in range(1,3):

        fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"

        single_location_irrradiance_da = loaded.isel(latitude=0, longitude=i).ground_irradiance
        latitude = single_location_irrradiance_da.latitude.values
        longitude = single_location_irrradiance_da.longitude.values

        single_location_tmy_df = weather.isel(gid=0).drop_vars(["Year", "Month", "Day", "Hour", "Minute", "gid"]).to_dataframe()

        conf_beds = irradiance_beds.testbeds_irradiance(
            irradiance_df=single_location_irrradiance_da.to_pandas(),
            conf=conf
        )

        
        
        

        #res = pd.concat([single_location_irrradiance_df, conf_beds, single_location_tmy_df], axis=1)

        #res.to_csv(f"beds-irradiance-{latitude}-{longitude}-conf-{conf}.csv")

In [29]:
xr.Dataset.from_dataframe(conf_beds).expand_dims(["latitude", "longitude"])

<xarray.Dataset> Size: 420kB
Dimensions:     (latitude: 1, longitude: 1, time: 8760)
Coordinates:
  * time        (time) datetime64[ns] 70kB 2001-01-01T00:30:00 ... 2001-12-31...
Dimensions without coordinates: latitude, longitude
Data variables:
    underpanel  (latitude, longitude, time) float64 70kB 0.0 0.0 0.0 ... 0.0 0.0
    edgetoedge  (latitude, longitude, time) float64 70kB 0.0 0.0 0.0 ... 0.0 0.0
    bedA        (latitude, longitude, time) float64 70kB 0.0 0.0 0.0 ... 0.0 0.0
    bedB        (latitude, longitude, time) float64 70kB 0.0 0.0 0.0 ... 0.0 0.0
    bedC        (latitude, longitude, time) float64 70kB 0.0 0.0 0.0 ... 0.0 0.0

In [20]:
conf_beds = irradiance_beds.testbeds_irradiance(
    irradiance_df=single_location_irrradiance_da.to_pandas(),
    conf=conf
)


NameError: name 'single_location_irrradiance_da' is not defined

In [10]:
conf = "01"
fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"

loaded = xr.open_dataset(fp)

irradiance_beds.output_template(loaded)

<xarray.Dataset> Size: 2GB
Dimensions:            (latitude: 25, longitude: 46, time: 8760,
                        distance_index: 10)
Coordinates:
  * latitude           (latitude) float32 100B 37.01 37.17 37.33 ... 40.69 40.85
  * longitude          (longitude) float32 184B -109.1 -108.9 ... -102.3 -102.2
  * distance_index     (distance_index) int64 80B 0 1 2 3 4 5 6 7 8 9
  * time               (time) datetime64[ns] 70kB 2001-01-01T00:30:00 ... 200...
Data variables: (12/16)
    temp_air           (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    dew_point          (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    dhi                (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    dni                (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    ghi                (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    albedo             (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    ...                 ...
    ground_irradiance  (time, distance_index, latitude, longitude) float64 806MB ...
    underpanel         (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    edgetoedge         (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    bedA               (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    bedB               (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0
    bedC               (latitude, longitude, time) float64 81MB 0.0 0.0 ... 0.0

In [ ]:
lats = (37.01,)
lons = (-108.9, -108.74)

single_location = xr.Dataset.from_dataframe(single_location_tmy_df).expand_dims({"latitude":lats, "longitude":lons})

single_location["ground_irradiance"] = single_location_irrradiance_da

single_location

In [ ]:
xr.Dataset(
    
    
)

In [ ]:
weather.isel(gid=0).drop_vars(["Year", "Month", "Day", "Hour", "Minute", "gid"])

In [6]:
fp = f"/projects/inspire/PySAM-MAPS/CO-sample/{conf}/1-16th-res-{conf}.nc"


loaded = xr.open_dataset(fp)

NameError: name 'conf' is not defined

In [7]:
loaded

,0.2502810919746036,0.7508432759238107,1.251405459873018,1.751967643822225,2.2525298277714323,2.753092011720639,3.2536541956698466,3.7542163796190535,4.2547785635682605,4.755340747517468
time,,,,,,,,,,
2001-01-01 00:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-01 01:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-01 02:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-01 03:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-01-01 04:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2001-12-31 19:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-12-31 20:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-12-31 21:30:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
### CONVERTING TO H5 from csvs
# this is a bad step, we do not want to do this on a large scale but CSV's existed already ehhh

import os
import glob
import xarray as xr

files = glob.glob(os.path.join(os.getcwd(), "beds-*.csv"))

In [ ]:
files

In [ ]:
from collections import defaultdict
# Group by configuration type (conf-01, conf-02, conf-03)
config_groups = defaultdict(list)

for file in files:
    # Extract configuration type from the filename
    parts = file.split("-")
    config_type = parts[-1].split(".")[0]  # Extract conf-01, conf-02, conf-03
    config_groups[config_type].append(file)

    
# Display the grouped results
for config, paths in sorted(config_groups.items()):
    print(f"{config} group has {len(paths)} locations:")
    for path in paths:
        print(f"  - {path}")
    print()

In [ ]:
config_groups

In [ ]:
for conf in ['01','02','03']:
    path_a = config_groups[conf][0]
    path_a = config_groups[conf][1]

    single = load_csv(path_a)

    # only contains first location for now
    builder = xr.Dataset.from_dataframe(single).expand_dims({"latitude":lats, "longitude":lons})
    
    second_location = xr.Dataset.from_dataframe(load_csv(path_b))
    
    builder = builder.assign({
        var: builder[var].where(~((builder.latitude == lats[0]) & (builder.longitude == lons[1])), second_location[var])
        for var in second_location.data_vars
    })

    builder.to_netcdf(f"{conf}-irradiance.h5", engine="h5netcdf")

    

In [ ]:
xr.open_dataset("03-irradiance.h5").edgetoedge.plot()

In [ ]:
files = groups['--108.73999786376953']
files = groups['--108.9000015258789']

# handle configs in order
files.sort()

path_a = files[0]
path_b = files[1]

def load_csv(path) -> xr.Dataset:

    single = pd.read_csv(path, index_col=0)
    single.index = pd.date_range("2001-01-01 00:30:00", periods=8760, freq="1h")
    single.index.name = "time"

    return single

single = load_csv(path_a)
single

In [ ]:
lats = [
37.0099983215332
]
    
lons = [
-108.73999786376953,
-108.9000015258789
]

# only contains first location for now
builder = xr.Dataset.from_dataframe(single).expand_dims({"latitude":lats, "longitude":lons})

second_location = xr.Dataset.from_dataframe(load_csv(path_b))

builder = builder.assign({
    var: builder[var].where(~((builder.latitude == lats[0]) & (builder.longitude == lons[1])), second_location[var])
    for var in second_location.data_vars
})

In [ ]:
builder.isel(latitude=0, longitude=0)